# MISA
Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

---

In [1]:
!pip install scikit-learn==1.1.3

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 237.1 kB/s eta 0:00:0000:0100:04


In [1]:
# AZA MANAMPY CODE ATO FA MNAOVA CELLULE VAOVAO

from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.linear_model import HuberRegressor
from sklearn.datasets import load_boston, load_diabetes, load_iris, load_digits
from scipy.special import huber
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
import warnings

warnings.simplefilter("ignore")

# Robust linear regression - Huber loss

In [32]:
np.random.seed(123)

In [33]:
data = load_boston()
X_train1, y_train1 = data.data, data.target
w1 = np.random.randn(X_train1.shape[1]) * 0.0001
b1 = np.random.randn(1) * 0.0001

In [34]:
def huber_loss_naive(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations

    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0

    N, D = X.shape

    for i in range(N):
        df = X[i] @ w + b - y[i]
        if np.abs(df) <= epsilon:
            loss += (df ** 2) / 2
            db += df
            for j in range(D):
                dw[j] += X[i, j] * df

        else:
            loss += epsilon * ( np.abs(df) - epsilon / 2)
            db += epsilon * np.sign(df)
            for j in range(D):
                dw[j] += X[i, j] * epsilon * np.sign(df)

    loss *= 1/N
    db *= 1/N
    dw *= 1/N

    loss += alpha * (w @ w) + alpha * b * b
    dw += 2 * alpha * w
    db += 2 * alpha * b

    return loss, dw, np.array(db).reshape(1,)

## without regularization

In [35]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -4.878257 analytic: -4.878257, relative error: 3.136290e-11
numerical: -15.340909 analytic: -15.340909, relative error: 1.344040e-11
numerical: -551.120158 analytic: -551.120158, relative error: 1.130019e-12
numerical: -4.878257 analytic: -4.878257, relative error: 3.136290e-11
numerical: -4.878257 analytic: -4.878257, relative error: 3.136290e-11
numerical: -0.093379 analytic: -0.093379, relative error: 3.488240e-10
numerical: -5.123308 analytic: -5.123308, relative error: 1.380354e-10
numerical: -4.878257 analytic: -4.878257, relative error: 3.136290e-11
numerical: -15.340909 analytic: -15.340909, relative error: 1.344040e-11
numerical: -551.120158 analytic: -551.120158, relative error: 1.130019e-12
numerical: -12.891700 analytic: -12.891700, relative error: 1.297658e-11
numerical: -551.120158 analytic: -551.120158, relative error: 1.130019e-12
numerical: -12.891700 analytic: -12.891700, relative error: 1.297658e-11
numerical: -0.093379 analytic: -0.093379

 ## with regularization

In [36]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -12.891446 analytic: -12.891446, relative error: 1.653018e-11
numerical: -0.748954 analytic: -0.748954, relative error: 1.520713e-11
numerical: -15.340710 analytic: -15.340710, relative error: 1.428239e-11
numerical: -92.576602 analytic: -92.576602, relative error: 2.547625e-12
numerical: -8.483926 analytic: -8.483926, relative error: 1.335535e-10
numerical: -481.509962 analytic: -481.509962, relative error: 2.806106e-13
numerical: -0.748954 analytic: -0.748954, relative error: 1.520713e-11
numerical: -481.509962 analytic: -481.509962, relative error: 2.806106e-13
numerical: -15.340710 analytic: -15.340710, relative error: 1.428239e-11
numerical: -0.093681 analytic: -0.093681, relative error: 4.217272e-10
numerical: -92.576602 analytic: -92.576602, relative error: 2.547625e-12
numerical: -0.093681 analytic: -0.093681, relative error: 4.217272e-10
numerical: -92.576602 analytic: -92.576602, relative error: 2.547625e-12
numerical: -92.576602 analytic: -92.5766

In [37]:
def huber_loss_vectorized(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations

    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0

    df = X @ w + b - y
    L = np.where(
            np.abs(df) <= epsilon,
            (df**2)/2,
            epsilon * ( np.abs(df) - epsilon / 2)
    )
    dL = np.where(
            np.abs(df) <= epsilon,
            df,
            epsilon * np.sign(df)
    )

    N, _ = X.shape
    
    loss = L.mean()
    db = dL.mean()
    dw = (X.T @ dL) / N

    
    loss += alpha * (w @ w) + alpha * b * b
    dw += 2 * alpha * w
    db += 2 * alpha * b

    return loss, dw, np.array(db).reshape(1,)

## without regularization

In [38]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -17.081635 analytic: -17.081635, relative error: 3.004335e-13
numerical: -8.484256 analytic: -8.484256, relative error: 4.187419e-15
numerical: -0.093379 analytic: -0.093379, relative error: 3.488238e-10
numerical: -481.509943 analytic: -481.509943, relative error: 2.476150e-13
numerical: -8.484256 analytic: -8.484256, relative error: 4.187419e-15
numerical: -481.509943 analytic: -481.509943, relative error: 2.476150e-13
numerical: -5.123308 analytic: -5.123308, relative error: 6.524417e-13
numerical: -0.748838 analytic: -0.748838, relative error: 9.981518e-11
numerical: -0.093379 analytic: -0.093379, relative error: 3.488238e-10
numerical: -8.484256 analytic: -8.484256, relative error: 4.187419e-15
numerical: -0.093379 analytic: -0.093379, relative error: 3.488238e-10
numerical: -551.120158 analytic: -551.120158, relative error: 1.590444e-13
numerical: -92.576117 analytic: -92.576117, relative error: 3.932019e-13
numerical: -24.914970 analytic: -24.914970, 

## with regularization

In [39]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -24.915106 analytic: -24.915106, relative error: 2.219171e-12
numerical: -12.891446 analytic: -12.891446, relative error: 2.749255e-12
numerical: -15.340710 analytic: -15.340710, relative error: 2.702855e-12
numerical: -481.509962 analytic: -481.509962, relative error: 2.809648e-13
numerical: -4.878474 analytic: -4.878474, relative error: 2.034902e-11
numerical: -8.483926 analytic: -8.483926, relative error: 2.542595e-12
numerical: -15.340710 analytic: -15.340710, relative error: 2.702855e-12
numerical: -24.915106 analytic: -24.915106, relative error: 2.219171e-12
numerical: -15.034595 analytic: -15.034595, relative error: 6.582209e-13
numerical: -4.878474 analytic: -4.878474, relative error: 2.034902e-11
numerical: -551.120331 analytic: -551.120331, relative error: 1.491427e-13
numerical: -8.483926 analytic: -8.483926, relative error: 2.542595e-12
numerical: -551.120331 analytic: -551.120331, relative error: 1.491427e-13
numerical: -5.123393 analytic: -5.12

In [40]:
class LinearModel():
    def __init__(self):
        self.w = None
        self.b = None

    def train(self, X, y, learning_rate=1e-3, alpha=0.0001, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape

        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w

        loss_history = []
        for it in range(num_iters):

            index = np.random.choice(N, batch_size, replace=False)
            X_batch = X[index, :]
            y_batch = y[index]

            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            self.w -= learning_rate * dw
            self.b -= learning_rate * db

            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class HuberRegression(LinearModel):
    """ Linear regression """

    def loss(self, X_batch, y_batch, alpha):
        return huber_loss_vectorized(self.w, self.b, X_batch, y_batch, alpha=alpha)

    def predict(self, X):
        return X @ self.w + self.b

In [41]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)

sk_model = HuberRegressor(fit_intercept=True)
sk_model.fit(X_train1, y_train1)
sk_pred = sk_model.predict(X_train1)
sk_mse = mean_squared_error(sk_pred, y_train1)

model = HuberRegression()
model.train(X_train1, y_train1, num_iters=75000, batch_size=64, learning_rate=1e-2, verbose=True)
pred = model.predict(X_train1)
mse = mean_squared_error(pred, y_train1)

print("MSE scikit-learn:", sk_mse)
print("MSE gradient descent model :", mse)
assert mse - sk_mse < 1

iteration 0 / 75000: loss 28.739515
iteration 10000 / 75000: loss 3.639797
iteration 20000 / 75000: loss 3.186416
iteration 30000 / 75000: loss 4.606638
iteration 40000 / 75000: loss 2.861491
iteration 50000 / 75000: loss 1.875911
iteration 60000 / 75000: loss 2.337350
iteration 70000 / 75000: loss 3.312214
MSE scikit-learn: 24.04102301055773
MSE gradient descent model : 24.340232992827627


# Multinomial logistic regression

In [42]:
data = load_iris()
X_train2, y_train2 = data.data, data.target

W = np.random.randn(X_train2.shape[1], 3) * 0.0001

In [43]:
def softmax_loss_naive(W, X, y, alpha):
    """
    Softmax loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """

    # Initialization
    loss = 0.0
    dW = np.zeros_like(W)
  
    C = len(np.unique(y))
    N, D = X.shape

    for i in range(N):
        pi = np.zeros(C)
        for k in range(C):
            pi[k] = X[i] @ W[:,k]
        pi = np.exp(pi)
        pi /= pi.sum()
        
        for k in range(C):
            loss -= (k == y[i]) * np.log(pi[k])
            for j in range(D):
                dW[j, k] -= X[i, j] * ((k == y[i]) - pi[k])
       
    loss += np.sum (alpha * W**2)
    dW += 2 * alpha * W
    
    return loss, dW

## Without regularization

In [44]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -41.309561 analytic: -41.309561, relative error: 2.867546e-11
numerical: 41.784076 analytic: 41.784076, relative error: 3.884906e-10
numerical: 114.741046 analytic: 114.741046, relative error: 6.266138e-11
numerical: -4.649518 analytic: -4.649518, relative error: 3.543856e-09
numerical: 4.214269 analytic: 4.214269, relative error: 1.134006e-10
numerical: -4.649518 analytic: -4.649518, relative error: 3.543856e-09
numerical: -89.628533 analytic: -89.628533, relative error: 6.150588e-11
numerical: 47.647283 analytic: 47.647283, relative error: 5.981568e-11
numerical: -89.628533 analytic: -89.628533, relative error: 6.150588e-11
numerical: -18.573172 analytic: -18.573172, relative error: 2.214723e-10
numerical: -89.628533 analytic: -89.628533, relative error: 6.150588e-11
numerical: 41.784076 analytic: 41.784076, relative error: 3.884906e-10


## With regularization

In [45]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -89.628719 analytic: -89.628719, relative error: 6.298188e-11
numerical: 14.358452 analytic: 14.358452, relative error: 2.514232e-10
numerical: 47.647356 analytic: 47.647356, relative error: 5.989268e-11
numerical: -25.112768 analytic: -25.112768, relative error: 3.961913e-10
numerical: 114.740862 analytic: 114.740862, relative error: 6.324210e-11
numerical: -25.112768 analytic: -25.112768, relative error: 3.961913e-10
numerical: 4.213474 analytic: 4.213474, relative error: 7.707101e-11
numerical: 114.740862 analytic: 114.740862, relative error: 6.324210e-11
numerical: -6.337892 analytic: -6.337892, relative error: 2.166521e-10
numerical: 41.783505 analytic: 41.783505, relative error: 3.862312e-10
numerical: 41.783505 analytic: 41.783505, relative error: 3.862312e-10
numerical: 4.213474 analytic: 4.213474, relative error: 7.707101e-11


In [46]:
def softmax_loss_vectorized(W, X, y, alpha, fit_intercept=False):
    """
    Softmax loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)
    
    C = len(np.unique(y))
    N, D = X.shape
    
    probs = np.exp(X @ W).T
    probs /= np.sum(probs, axis=0)
    
    loss -= np.sum(np.log(probs.T[np.ix_(np.arange(N), y)].diagonal()))
    dW += X.T @ (probs.T - np.eye(C)[y, :])
    
    loss += np.sum (alpha * W**2)
    dW += 2 * alpha * W
    
    return loss, dW

## Without regularization

In [47]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -89.628533 analytic: -89.628533, relative error: 9.321636e-11
numerical: -37.134559 analytic: -37.134559, relative error: 5.350852e-10
numerical: -37.134559 analytic: -37.134559, relative error: 5.350852e-10
numerical: -25.112512 analytic: -25.112512, relative error: 2.864185e-10
numerical: -37.134559 analytic: -37.134559, relative error: 5.350852e-10
numerical: -89.628533 analytic: -89.628533, relative error: 9.321636e-11
numerical: 41.784076 analytic: 41.784076, relative error: 4.905214e-10
numerical: 114.741046 analytic: 114.741046, relative error: 7.504654e-11
numerical: -6.337722 analytic: -6.337722, relative error: 1.881496e-10
numerical: -4.649518 analytic: -4.649518, relative error: 4.307960e-09
numerical: 47.647283 analytic: 47.647283, relative error: 1.654548e-13
numerical: 114.741046 analytic: 114.741046, relative error: 7.504654e-11


## With regularization

In [48]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -37.134198 analytic: -37.134198, relative error: 5.389547e-10
numerical: 4.213474 analytic: 4.213474, relative error: 4.143409e-10
numerical: 41.783505 analytic: 41.783505, relative error: 4.882635e-10
numerical: 47.647356 analytic: 47.647356, relative error: 2.425523e-13
numerical: -89.628719 analytic: -89.628719, relative error: 9.469229e-11
numerical: 14.358452 analytic: 14.358452, relative error: 2.019369e-10
numerical: -6.337892 analytic: -6.337892, relative error: 1.196787e-10
numerical: 4.213474 analytic: 4.213474, relative error: 4.143409e-10
numerical: 114.740862 analytic: 114.740862, relative error: 7.562727e-11
numerical: 47.647356 analytic: 47.647356, relative error: 2.425523e-13
numerical: 4.213474 analytic: 4.213474, relative error: 4.143409e-10
numerical: 47.647356 analytic: 47.647356, relative error: 2.425523e-13


## Gradient descent

In [49]:
class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            X =  np.hstack( (np.ones((X.shape[0],1)), X) )

        N, d = X.shape

        C = (np.max(y) + 1)
        if self.W is None: # Initialization
            self.W = 0.001 * np.random.randn(d, C)

        # Run stochastic gradient descent to optimize W

        loss_history = []
        for it in range(num_iters):
            
            index = np.random.choice(N, batch_size, replace=False)
            X_batch = X[index, :]
            y_batch = y[index]

            # evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            self.W -= learning_rate * dW

            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class MultinomialLogisticRegressor(LinearModel):
    """ Softmax regression """

    def loss(self, X_batch, y_batch, alpha):
        return softmax_loss_vectorized(self.W, X_batch, y_batch, alpha)

    def predict(self, X):
        """
        Inputs:
        - X: array of shape (N, D)

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class
        """
        if self.fit_intercept:
            X =  np.hstack( (np.ones((X.shape[0],1)), X) )
            
        y_pred = np.argmax(X @ self.W, axis=1)
        return y_pred

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)

sk_model = LogisticRegression(fit_intercept=False)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=False)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.01

In [51]:
sk_model = LogisticRegression(fit_intercept=True)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=True)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.02

iteration 0 / 75000: loss 70.309919
iteration 10000 / 75000: loss 4.283092
iteration 20000 / 75000: loss 2.773059
iteration 30000 / 75000: loss 1.718742
iteration 40000 / 75000: loss 2.054056
iteration 50000 / 75000: loss 3.970647
iteration 60000 / 75000: loss 4.078693
iteration 70000 / 75000: loss 2.061785
Accuracy scikit-learn: 0.9733333333333334
Accuracy gradient descent model : 0.9866666666666667


# K-Nearest Neighbor

## Computing distances

In [52]:
data = load_digits()
X_train3, y_train3 = data.data, data.target
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_train3, y_train3, test_size=0.33, random_state=2)

def get_distances_two_loops_with_norm(X_train, X_test):
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            distances[i, j] = np.linalg.norm(X_test[i] - X_train[j])
    return distances

In [53]:
def get_distances_two_loops(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train

    Inputs:
    - X_test: array of shape (num_test, D)

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between
    the ith test point and the jth training point.
    """
    num_test, D = X_test.shape
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            for k in range(D):
                distances[i, j] += (X_train[j, k] - X_test[i, k])**2
            distances[i, j] = np.sqrt(distances[i, j])
    return distances

In [54]:
distances = get_distances_two_loops(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


In [55]:
def compute_distances_one_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train

    Inputs:
    - X_test: array of shape (num_test, D)

    Returns:
    - dists: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    
    for i in range(num_test):
        distances[i,:] = np.sqrt(np.sum((X_test[i] - X_train)**2, axis=1))
    return distances

In [56]:
distances = compute_distances_one_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


In [188]:
def get_distances_zero_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train

    Inputs:
    - X_test: array of shape (num_test, D)

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    
    
    distances = np.sqrt(np.sum((X_test[:, np.newaxis] - X_train)**2, axis=2))

    return distances

In [189]:
distances = get_distances_zero_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


## K-Nearest Neighbor (knn) classifier

In [190]:
class KNearestNeighborClassifier():
    """ kNN classifier using L2 distance """

    def __init__(self, k=1):
        """
        Inputs:
        - k: number of nearest neighbors that vote for the predicted labels.
        """
        self.k = k

    def fit(self, X, y):
        """
        Train the classifier. Just memorize the training data.

        Inputs:
        - X: array of shape (num_train, D)
        - y: array of shape (N,)
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        """
        Predict labels for test data using this classifier.

        Inputs:
        - X: array of shape (num_test, D)

        Returns:
        - y: array of shape (num_test,)
        """
        distances = get_distances_zero_loop(self.X_train, X)
        return self.predict_labels(distances)

    def predict_labels(self, distances):
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.

        Inputs:
        - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between
        the ith test point and the jth training point.

        Returns:
        - y:  array of shape (num_test,)
        """
        num_test = distances.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            # list storing the labels of the k nearest neighbors to the ith test point.
            closest_y = []

            idx = np.argsort(distances[i,:])
            
            closest_y = self.y_train[idx[:self.k]]
            
            unq, count = np.unique(closest_y, return_counts=True)
            most = np.max(count)
            
            y_pred[i] = np.min(unq[count == most])
            

        return y_pred

In [191]:
sk_model = KNeighborsClassifier(n_neighbors=3)
sk_model.fit(X_train3, y_train3)
sk_pred = sk_model.predict(X_test3)
sk_accuracy = accuracy_score(y_test3, sk_pred)

model = KNearestNeighborClassifier(k=3)
model.fit(X_train3, y_train3)
pred = model.predict(X_test3)
model_accuracy = accuracy_score(y_test3, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 1e-10

Accuracy scikit-learn: 0.9831649831649831
Accuracy gradient descent model : 0.9831649831649831


## cross-validation

In [192]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []

# Split up the data into folds
# X_train_folds and y_train_folds lits of length num_folds

N = X_train3.shape[0]

l = N // num_folds

for i in range(num_folds):
    X_train_folds.append(X_train3[(i*l):(i*l)+l])
    y_train_folds.append(y_train3[(i*l):(i*l)+l])
    
# A dictionary of length num_folds holding the accuracies for different values of k
k_to_accuracies = {}

for k in k_choices:
    m = KNearestNeighborClassifier(k)
    accuracies = []
    for n in range(num_folds):
        xtr, xvl = np.delete(X_train_folds, n, axis=0), X_train_folds[n]
        ytr, yvl = np.delete(y_train_folds, n, axis=0), y_train_folds[n]

        xtr = np.concatenate(xtr)
        ytr = np.concatenate(ytr)
        
        m.fit(xtr, ytr)
        pred = m.predict(xvl)
        accuracies.append(accuracy_score(yvl, pred))

    k_to_accuracies[k] = accuracies

In [193]:
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

k = 1, accuracy = 0.991667
k = 1, accuracy = 0.983333
k = 1, accuracy = 0.983333
k = 1, accuracy = 0.970833
k = 1, accuracy = 0.995833
k = 3, accuracy = 0.987500
k = 3, accuracy = 0.983333
k = 3, accuracy = 0.991667
k = 3, accuracy = 0.970833
k = 3, accuracy = 0.983333
k = 5, accuracy = 0.991667
k = 5, accuracy = 0.983333
k = 5, accuracy = 0.991667
k = 5, accuracy = 0.979167
k = 5, accuracy = 0.983333
k = 8, accuracy = 0.979167
k = 8, accuracy = 0.970833
k = 8, accuracy = 0.987500
k = 8, accuracy = 0.966667
k = 8, accuracy = 0.983333
k = 10, accuracy = 0.983333
k = 10, accuracy = 0.970833
k = 10, accuracy = 0.979167
k = 10, accuracy = 0.966667
k = 10, accuracy = 0.975000
k = 12, accuracy = 0.983333
k = 12, accuracy = 0.975000
k = 12, accuracy = 0.979167
k = 12, accuracy = 0.966667
k = 12, accuracy = 0.979167
k = 15, accuracy = 0.983333
k = 15, accuracy = 0.962500
k = 15, accuracy = 0.983333
k = 15, accuracy = 0.966667
k = 15, accuracy = 0.962500
k = 20, accuracy = 0.966667
k = 20, accu